In [1]:
!ls ../models_library/raw_models/

DunedinPoAm.csv			 McCartneyBlood_2018_BMI.csv
EpiTOC2.csv			 McCartneyBlood_2018_Education.csv
HannumBlood_2013.csv		 McCartneyBlood_2018_HDL.csv
HannumBreast_2013.csv		 McCartneyBlood_2018_LDL.csv
HannumKidney_2013.csv		 McCartneyBlood_2018_Smoking.csv
HannumLung_2013.csv		 McCartneyBlood_2018_TC.csv
HorvathMultiShrunken_2013.csv	 McCartneyBlood_2018_TotalFat.csv
HorvathMulti_2013.csv		 McCartneyBlood_2018_WaistTipRatio.csv
KnightBlood_2016.csv		 PhenoAge.csv
LinBlood3CpG_2016.csv		 YingAdaptAge.csv
LinBlood99CpG_2016.csv		 YingCausAge.csv
McCartneyBlood_2018_Alcohol.csv  YingDamAge.csv


In [1]:
import numpy as numpy
import pandas as pd 


In [2]:
import sys
import os


data_library = os.path.abspath('../data_library')
sys.path.append(data_library)

from data_repository import import_data




In [3]:
! ls ../data_library/

__init__.py  __pycache__  data_repository.py


In [4]:
SAMPLE_ID = 'GSE41169_series_matrix.txt'
URL = 'https://drive.google.com/file/d/1cZca-FHnfeUerXJOzqhnH34mUYjNOgba/view?usp=sharing'
data_test = import_data(SAMPLE_ID, URL)


Downloading...
From (original): https://drive.google.com/uc?id=1cZca-FHnfeUerXJOzqhnH34mUYjNOgba
From (redirected): https://drive.google.com/uc?id=1cZca-FHnfeUerXJOzqhnH34mUYjNOgba&confirm=t&uuid=f158dd4f-dbdb-4828-8b51-c6450a5612c9
To: /home/aryuna/bioinf_inst/project/ComputAge/computage/deage/GSE41169_series_matrix.txt
100%|██████████| 477M/477M [00:41<00:00, 11.5MB/s] 


In [5]:
data_test.to_csv('../data_library/data_test.csv', sep = '\t', index=False)


In [7]:
model_filepath = '../models_library/raw_models/HannumLung_2013.csv'

model_data = pd.read_csv(model_filepath)
features = model_data[['Feature_ID']]
coefficients = model_data[['Coef']]


In [12]:
data_test

,0,1,2
0,"""ID_REF""","""GSM1009660""","""GSM1009661"""
1,"""cg00000029""",0.4549308,0.4102866
2,"""cg00000108""",0.9078832,0.9158939
3,"""cg00000109""",0.8627719,0.8809996
4,"""cg00000165""",0.1604464,0.1375998
...,...,...,...
485573,"""rs9363764""",0.530962,0.9453855
485574,"""rs939290""",0.5073323,0.4720046
485575,"""rs951295""",0.04051992,0.03988533
485576,"""rs966367""",0.9235626,0.03282508


In [28]:
df = data_test.copy()
df.columns = df.iloc[0]
df.columns = df.columns.str.replace('"', '')
df = df[1:]
df = df.reset_index(drop=True)
df.ID_REF = df.ID_REF.str.replace('"', '')


In [29]:
df.head()

,ID_REF,GSM1009660,GSM1009661
0,cg00000029,0.4549308,0.4102866
1,cg00000108,0.9078832,0.9158939
2,cg00000109,0.8627719,0.8809996
3,cg00000165,0.1604464,0.1375998
4,cg00000236,0.7281801,0.7367078


In [11]:
features

,Feature_ID
0,cg23606718
1,cg10470891
2,cg16867657
3,cg24724428
4,cg20442599
5,cg21186299
6,cg21205222
7,cg08715791
8,cg11354629
9,cg03036557
